# LocalCart scenario part 4: Visualize streaming data in a real-time dashboard

In this fourth notebook in the series, you'll install and configure a Node.js app and deploy it to IBM Bluemix to act as a web server. The Node.js app will be the real-time dashboard that presents a web interface to each visitor showing a real-time of view of the streaming data.

The Node.js app will fetch data from the Compose for Redis database that you created in the second notebook of the series, and dispatch that data to the connected front-end to visualize the streaming data. By the end of the notebook, you'll understand how to deploy a dashboard app to IBM Bluemix to visualize streaming data, and how to simulate streaming data if you don't have a streaming data source.

This notebook runs on Python 2 with Spark 2.0.

## Table of contents

1. [Setup](#setup)<br>
2. [Load sample data](#load)<br>
3. [Install and deploy dashboard app to Bluemix](#install_deploy)<br>
    3.1 [Install and deploy the dashboard app automatically](#install_auto)<br>
    3.2 [Install and deploy the dashboard app manually](#install_manually)<br>
    3.3 [Install the dashboard app locally](#install_locally)<br>
4. [Summary and next steps](#summary)<br>

<a id="setup"></a>
## 1. Setup

Before you use the example code in this notebook, follow these setup steps:

1. Log in to  <a target="_blank" href="https://console.ng.bluemix.net/">IBM Bluemix</a> and go to the space in your Bluemix account that you created in notebook 1. (ADD LINK TO NOTEBOOK 1)
1. <a target="_blank" href="https://github.com/cloudfoundry/cli#downloads">Download CloudFoundry CLI</a> on your local machine. 
1. Ensure that you have created the following components:
    * The source of data piped into MessageHub topics that you set up in the first notebook in the series. (ADD LINK TO NOTEBOOK 1)
    * The Streaming Pipelines service that you created in the second notebook of the series(ADD LINK TO NOTEBOOK 2). The Streaming Pipelines service subscribes to the MessageHub topics and aggregates the data into the Compose for Redis database.
    * A Compose for Redis database that you set up in the second notebook of the series to store the aggregated data prior to it being rendered. (ADD LINK TO NOTEBOOK 2) 
    
    If you did not run notebook 2 of the series, you must create a Compose for Redis database. You can use the <a target="_blank" href="https://console.bluemix.net/catalog/services/compose-for-redis">Compose for Redis</a> or the <a target="_blank" href="https://console.bluemix.net/catalog/services/redis-cloud">Redis Cloud</a> services in IBM Bluemix. To connect to the Compose for Redis database, you need the following information:
        * Redis password
        * Host name 
        * Host port
             
1. If you have not run notebooks 1 and 2 of the series, go to the [Load sample data](#load) section, to use a Python script that will simulate data for you. 

<a id="load"></a>
## 2. Load sample data

If you have not run notebook 2 of the series, you need a source of streaming data. 

Replace `<host_name>`, `<host_port>` and `<redis_password>` with your Redis database's credentials and then run the following cell to simulate some data for the dashboard app:

In [1]:
!pip install redis
import redis
import time

i = 1
while True:
    r = redis.StrictRedis(host='<host_name>', port=<host_port>, db=0, password='<redis_password>')
    i = i + 1
    r.hset('funnel', 'basket_count', 3*i);
    r.hset('funnel', 'basket_total', 2*i);
    r.hset('funnel', 'checkout_count', i);
    r.hset('funnel', 'checkout_total', i*75);
    r.hset('funnel', 'login_count', 5*i);
    time.sleep(2)       
    if (i > 100):
        print 'Simulation complete'
        break;

Simulation complete


<a id="install_deploy"></a>
## 3. Install and deploy dashboard app to Bluemix

You can install the dashboard app and deploy it to IBM Bluemix either automatically through the click of a button, manually or locally.

<a id="install_auto"></a>
### 3.1. Install and deploy the dashboard app automatically

To install the app automatically:

1. Click the following button: <a target="_blank" href="https://bluemix.net/deploy?repository=https://github.com/ibm-watson-data-lab/advo-beta-dashboard">
    <img src="https://deployment-tracker.mybluemix.net/stats/aa7ed2e962f16e8eee0b5ee0ab5f31fb/button.svg">
</a> <br/>
This button installs the Node.js app that will act as the real-time dashboard to visualize the streaming data. The code of this Node.js app is open-source and published on this  <a target="_blank" href="https://github.com/ibm-watson-data-lab/advo-beta-dashboard">GitHub repository</a>.
1. Click **Deploy** to deploy the app to Bluemix.
1. Go to the Bluemix dashboard, select the app you just created, and click the **Runtime** tab.
1. In the **Environment variables** tab, create a custom environment variable called `REDIS_URL` with the URL of your Compose for Redis instance. The URL used as value for the `REDIS_URL` environment variable looks as follows: 

    ```      
    redis://x:<user_password>@<host_name>:<host_port>     
    ```
 
 <img src='https://github.com/notebookgraphics/advobeta/blob/master/env_variables_value.PNG?raw=true' width='1500'></img> <br>
   
1. Restart the app and visit the URL to see the dashboard in action.

<img src='https://github.com/notebookgraphics/advobeta/blob/master/dashboardapp.gif?raw=true' width='1500'></img>

<a id="install_manually"></a>
### 3.2. Install and deploy the dashboard app manually

You can choose to modify the app code yourself then install it and deploy it manually from the public GitHub repository:

1. Fork the <a target="_blank" href="https://github.com/ibm-watson-data-lab/advo-beta-dashboard#fork-destination-box">advo-beta-dashboard GitHub repository</a> into your own account.
1. Clone the forked repository to your own machine.
1. Install the <a target="_blank" href="https://console.bluemix.net/docs/cli/index.html">Bluemix cfpush command-line utility</a>.
1. Change the app code as needed.
1. Run `cf push` from the repository's directory to deploy the app to Bluemix.

<a id="install_locally"></a>
### 3.3. Install the dashboard app locally

You can choose to run the dashboard on your own machine with <a target="_blank" href="https://nodejs.org/en/download/">Node.js</a>[]() installed. To do so, add the Compose for Redis credentials and run the following commands:

```sh
# clone the code
git clone https://github.com/ibm-watson-data-lab/advo-beta-dashboard

# change directory
cd advo-beta-dashboard

# install dependencies
npm install

# insert your own Redis URL here
export REDIS_URL="redis://x:<user_password>@<host_name>:<host_port>"

# run the dashboard
npm start
```

At the end of the setup, the dashboard app displays the port that the app is using, for example:

```
Dashboard app listening on port 6039
```

You must simply go to the http://localhost:6039 in your web browser to access the dashboard app and visualize the streaming data.

<a id="summary"></a>
## 4. Summary and next steps

You learned how deploy a real-time dashboard app in Bluemix to visualize streaming data that is fetched from a Compose for Redis database. You also learned to how create sample data to see the real-time dashboard in action.

Check out other notebooks in this series: 
 - Generating a Kafka producer (JSON) into MessageHub. (need link when published)
 - Building the Streaming Pipeline (need link when published) 
 - Recommendation Engine API (need link when published) 

### Author
Glynn Bird is a developer advocate at IBM's Watson Data Platform. 

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.